In [22]:
import pandas as pd
import numpy as np
import json
import urllib
import urllib.request
import requests
import datetime
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [74]:
# datos sobre latitud, longitud, y altitud de cada provincia
# fuente: https://www.ign.es/web/ane-datos-geograficos/-/datos-geograficos/datosPoblacion?tipoBusqueda=capitales

url='https://www.ign.es/web/ane-datos-geograficos/-/datos-geograficos/datosPoblacion?tipoBusqueda=capitales'
h = urllib.request.urlopen(url)
soup = BeautifulSoup(h)
trs = soup.find_all('tr')
provs = []
lats = []
lngs = []
altitud = []
for el in trs[3:]:
    for i, td in enumerate(el):
        #print(i, td)
        if i == 3:
            provs.append(td.text)
        if i == 7:
            lats.append(float(td.text.replace(',', '.')))
        if i == 9:
            lngs.append(float(td.text.replace(',', '.')))
        if i == 15:
            altitud.append(int(td.text.split('.')[0]))
df = pd.DataFrame(columns = ['province', 'latitude', 'longitude', 'altura'])
df['province'] = provs
df['latitude'] = lats
df['longitude'] = lngs
df['altitud'] = altitud
df.province.replace({'Las Palmas': 'Palmas, Las', 'La Rioja': 'Rioja, La', 'A Coruña': 'Coruña, A', 'Illes Balears': 'Balears, Illes',
                    'Álava': 'Araba/Álava', 'Alicante / Alacant':'Alicante/Alacant', 'Castellón / Castelló': 'Castellón/Castelló',
                    'Valencia / València': 'Valencia', 'Valencia / Valènci': 'Valencia', 'Valencia/Valènci': 'Valencia'}, inplace=True)

In [75]:
df

,province,latitude,longitude,altura,altitud
0,"Coruña, A",43.370126,-8.391149,NaN,21
1,Albacete,38.995881,-1.855747,NaN,681
2,Alicante/Alacant,38.345487,-0.483183,NaN,5
3,Almería,36.838924,-2.464132,NaN,16
4,Ávila,40.655870,-4.697713,NaN,1
5,Badajoz,38.878743,-6.970997,NaN,182
6,Barcelona,41.384247,2.176349,NaN,13
7,Vizcaya,43.257220,-2.923906,NaN,6
8,Burgos,42.341130,-3.704198,NaN,859
9,Cáceres,39.473168,-6.371211,NaN,457


In [6]:
# diccionario para unir provincias con sus comunidades
coms_dict = {'Almería': 'Andalucía',
 'Cádiz': 'Andalucía',
 'Córdoba': 'Andalucía',
 'Granada': 'Andalucía',
 'Huelva': 'Andalucía',
 'Jaén': 'Andalucía',
 'Málaga': 'Andalucía',
 'Sevilla': 'Andalucía',
 'Huesca': 'Aragón',
 'Teruel': 'Aragón',
 'Zaragoza': 'Aragón',
 'Oviedo': 'Asturias',
 'Palma de Mallorca': 'Baleares',
 'Santa Cruz de Tenerife': 'Canarias',
 'Las Palmas de Gran Canaria': 'Canarias',
 'Santander': 'Cantabria',
 'Albacete': 'Castilla-La Mancha',
 'Ciudad Real': 'Castilla-La Mancha',
 'Cuenca': 'Castilla-La Mancha',
 'Guadalajara': 'Castilla-La Mancha',
 'Toledo': 'Castilla-La Mancha',
 'Ávila': 'Castilla y León',
 'Burgos': 'Castilla y León',
 'León': 'Castilla y León',
 'Salamanca': 'Castilla y León',
 'Segovia': 'Castilla y León',
 'Soria': 'Castilla y León',
 'Valladolid': 'Castilla y León',
 'Zamora': 'Castilla y León',
 'Barcelona': 'Cataluña',
 'Gerona': 'Cataluña',
 'Lérida': 'Cataluña',
 'Tarragona': 'Cataluña',
 'Alicante': 'Comunidad Valenciana',
 'Castellón de la Plana': 'Comunidad Valenciana',
 'Valencia': 'Comunidad Valenciana',
 'Badajoz': 'Extremadura',
 'Cáceres': 'Extremadura',
 'La Coruña': 'Galicia',
 'Lugo': 'Galicia',
 'Orense': 'Galicia',
 'Pontevedra': 'Galicia',
 'Madrid': 'Madrid',
 'Murcia': 'Murcia',
 'Pamplona': 'Navarra',
 'Bilbao': 'País Vasco',
 'San Sebastián': 'País Vasco',
 'Vitoria': 'País Vasco',
 'Logroño': 'La Rioja',
 'Coruña, A': 'Galicia',
 'Asturias': 'Asturias',
 'Cantabria': 'Cantabria',
 'Castellón/Castelló': 'Comunidad Valenciana',
 'Ceuta': 'Ceuta',
 'Girona': 'Cataluña',
 'Balears, Illes': 'Balears, Illes',
 'Rioja, La': 'Rioja, La',
 'Palmas, Las': 'Canarias',
 'Lleida': 'Cataluña',
 'Navarra': 'Navarra',
 'Ourense': 'Galicia',
 'Palencia': 'Castilla y León',
 'Valencia/València': 'Comunidad Valenciana',
            'Valencia': 'Comunidad Valenciana'}

In [7]:
# Añadir columna de comunidad a df
df['comunidad'] = df.province.replace(coms_dict)

In [8]:
# Provincias de España y sus URLs para scrapear los datos de AQI y tiempo de iqair.com.

spain_urls = {'Albacete': 'https://www.iqair.com/spain/castille-la-mancha/albacete',
 'Alicante/Alacant': 'https://www.iqair.com/spain/valencia/alacant',
 'Almería': 'https://www.iqair.com/spain/andalucia/almeria',
 'Araba/Álava': '',
 'Asturias': 'https://www.iqair.com/spain/asturias/oviedo',
 'Ávila': 'https://www.iqair.com/spain/castille-and-leon/avila',
 'Badajoz': 'https://www.iqair.com/spain/extremadura/badajoz',
 'Balears, Illes': 'https://www.iqair.com/spain/balearic-islands/ibiza',
 'Barcelona': 'https://www.iqair.com/spain/catalunya/barcelona',
 'Bizkaia': '',
 'Burgos': 'https://www.iqair.com/spain/castille-and-leon/burgos',
 'Cáceres': 'https://www.iqair.com/spain/extremadura/caceres',
 'Cádiz': 'https://www.iqair.com/spain/andalucia/cadiz',
 'Cantabria': 'https://www.iqair.com/spain/cantabria/santander',
 'Castellón/Castelló': 'https://www.iqair.com/spain/valencia/castello-de-la-plana',
 'Ciudad Real': 'https://www.iqair.com/spain/castille-la-mancha/ciudad-real',
 'Córdoba': 'https://www.iqair.com/spain/andalucia/cordoba',
 'Coruña, A': 'https://www.iqair.com/spain/galicia/a-coruna',
 'Cuenca': 'https://www.iqair.com/spain/castille-la-mancha/cuenca',
 'Gipuzkoa': '',
 'Girona': 'https://www.iqair.com/spain/catalunya/girona',
 'Granada': 'https://www.iqair.com/spain/andalucia/granada',
 'Guadalajara': 'https://www.iqair.com/spain/castille-la-mancha/guadalajara',
 'Huelva': 'https://www.iqair.com/spain/andalucia/huelva',
 'Huesca': 'https://www.iqair.com/spain/aragon/huesca',
 'Jaén': 'https://www.iqair.com/spain/andalucia/jaen',
 'León': 'https://www.iqair.com/spain/castille-and-leon/leon',
 'Lleida': 'https://www.iqair.com/spain/catalunya/lleida',
 'Lugo': 'https://www.iqair.com/spain/galicia/lugo',
 'Madrid': 'https://www.iqair.com/spain/madrid',
 'Málaga': 'https://www.iqair.com/spain/andalucia/malaga',
 'Murcia': 'https://www.iqair.com/spain/murcia',
 'Navarra': 'https://www.iqair.com/spain/navarre/pamplona',
 'Ourense': 'https://www.iqair.com/spain/galicia/ourense',
 'Palencia': 'https://www.iqair.com/spain/castille-and-leon/palencia',
 'Palmas, Las': 'https://www.iqair.com/spain/canarias/las-palmas-de-gran-canaria',
 'Pontevedra': '',
 'Rioja, La': 'https://www.iqair.com/spain/la-rioja/logrono',
 'Salamanca': 'https://www.iqair.com/spain/castille-and-leon/salamanca',
 'Santa Cruz de Tenerife': 'https://www.iqair.com/spain/canarias/santa-cruz-de-tenerife',
 'Segovia': 'https://www.iqair.com/spain/castille-and-leon/segovia',
 'Sevilla': 'https://www.iqair.com/spain/andalucia/sevilla',
 'Soria': 'https://www.iqair.com/spain/castille-and-leon/soria',
 'Tarragona': 'https://www.iqair.com/spain/catalunya/tarragona',
 'Teruel': 'https://www.iqair.com/spain/aragon/teruel',
 'Toledo': 'https://www.iqair.com/spain/castille-la-mancha/toledo',
 'Valencia/València': 'https://www.iqair.com/spain/valencia',
 'Valladolid': 'https://www.iqair.com/spain/castille-and-leon/valladolid',
 'Zamora': 'https://www.iqair.com/spain/castille-and-leon/zamora',
 'Zaragoza': 'https://www.iqair.com/spain/aragon/zaragoza',
 'Ceuta': 'https://www.iqair.com/spain/ceuta',
 'Melilla': ''}

In [12]:
# función para sacar datos de tiempo y contaminación del aire en las provincias de España

def make_dict_provincia(url, city):
    try:
        h = urllib.request.urlopen(url)
    except:
        print(f"{city}, {url} not found")
    soup = BeautifulSoup(h)
    pollutants = ['PM2.5', 'PM10', 'O3', 'NO2', 'SO2', 'CO']
    date = datetime.today()
    province_dict = {}
    province_dict['City'] = city
    test_list = []
    for t in soup.find_all('td'):
        for hijo in t:
            test_list.append(hijo)
    for i in range(len(test_list[:9])):
        if i%2==0:
            province_dict[test_list[i]]=test_list[i+1]
    aqi = None
    for i in range(len(test_list)): 
        if 'AQI' in str(test_list[i]) or 'ICA EE.UU.' in str(test_list[i]):
            try:
                aqi = int(str(test_list[i-1]).replace('*', '').strip())
            except:
                pass
    province_dict['AQI']=aqi
    for p in pollutants:
        province_dict[p] = 'Unknown'
    for el in soup.find_all('table', class_="aqi-overview-detail__other-pollution-table"):
        for hijo in el:
            answers = hijo.text
    answers = answers.split()
    for i in range(len(answers)-1):
        if i%2 == 0:
            province_dict[answers[i]] = answers[i+1]
    province_dict['Date'] = date
    return province_dict

def make_province_df():
    dict_all = {}
    for prov, url in spain_urls.items():
        if len(url)> 2:
            dict_all[prov] = make_dict_provincia(url, prov)
            df = pd.DataFrame.from_dict(dict_all, orient='index')
    return df

In [65]:
# función para sacar muchos datos de contaminantes y tiempo. 
# Al final, no utilicé esta función porque decidí analizar solo provincias en lugar de muchas ciudades en diferentes países.

def predict_everything(url, city, country):
    h = urllib.request.urlopen(url)
    soup = BeautifulSoup(h)
    past_aqi = soup.find_all('b')
    aqis = [a.text for a in past_aqi[1:]]
    Days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Today']
    h = urllib.request.urlopen(url)
    soup = BeautifulSoup(h)
    low_temps = soup.find_all('span')
    lows = []
    high_temps = soup.find_all('strong')
    highs = []
    dates = soup.find_all('td')
    day = []
    wind1 = soup.find_all('p')
    wind = []
    for t in wind1:
        if 'km/h' in t.text:
            new_t = t.text.replace('km/h', '')
            wind.append(float(new_t))
    for p in dates:
        for d in Days:
            if d in p.text:
                day.append(p.text)
    for t in low_temps:
        if '°' in t.text:
            new_t = t.text.replace('°', '')
            lows.append(int(new_t))
    for t in high_temps:
        new_t = t.text.replace('°', '')
        try:
            highs.append(int(new_t))
        except:
            pass
    date = [datetime.today() for i in range(len(aqis))]
    df = pd.DataFrame(columns=['day', 'aqi', 'temp_high', 'temp_low', 'wind_speed', 'city', 'today'])
    if len(highs) < len(aqis):
        for i in range(len(aqis)-len(highs)):
            highs.append(None)
    if len(lows) < len(aqis):
        for i in range(len(aqis)-len(lows)):
            lows.append(None)
    if len(wind) < len(aqis):
        for i in range(len(aqis)-len(wind)):
            wind.append(None)
    df['day'] = day
    df['aqi'] = aqis
    df['temp_high'] = highs
    df['temp_low'] = lows
    df['wind_speed'] = wind
    df['city'] = city
    df['today'] = date
    df['today'] = [i.floor('H') for i in df['today']]
    polluted_cities = soup.find_all('td')
    city_list = []
    neighborhood = []
    aqi_city = []
    aqi_neighborhood = []
    date2 = [datetime.today() for i in range(len(city_list))]
    for p in range(10, 40):
        if p%3 == 0:
            aqi_city.append(polluted_cities[p].text.strip())
            aqi_neighborhood.append(polluted_cities[p+30].text.strip())
        elif p%3 == 2:
            city_list.append(polluted_cities[p].text.strip())
            neighborhood.append(polluted_cities[p+30].text.strip())
    date2 = [datetime.today() for i in range(len(city_list))]
    df_top_polluted = pd.DataFrame(columns = ['city', 'aqi', 'date', 'country'])
    df_neighborhoods = pd.DataFrame(columns = ['neighborhood', 'aqi', 'date', 'city'])
    df_top_polluted['city'] = city_list
    df_neighborhoods['neighborhood'] = neighborhood
    df_top_polluted['aqi'] = aqi_city
    df_neighborhoods['aqi'] = aqi_neighborhood
    df_top_polluted['date'] = date2
    df_top_polluted['date'] = [i.floor('H') for i in df_top_polluted['date']]
    df_neighborhoods['date'] = df_top_polluted['date'][:len(neighborhood)]
    df_neighborhoods['city'] = [city for i in range(len(neighborhood))]
    df_top_polluted['country'] = [country for i in range(len(aqi_city))]
    return df, df_top_polluted, df_neighborhoods

In [13]:
# Utilicé el código de abajo para sacar los datos de AQI y tiempo varias veces al día para tener datos suficientes.
spain_provinces_df = make_province_df()
spain_provinces_df.rename(columns={'PM2.5':'PM2_5'}, inplace=True)
spain_provinces_df.rename(columns={'City':'Provincia'}, inplace=True)

In [16]:
spain_provinces_df.sample()

,Provincia,Weather,Temperature,Humidity,Wind,Pressure,AQI,PM2_5,PM10,O3,NO2,SO2,CO,Date
Tarragona,Tarragona,Few clouds,18°C,61%,24.1 km/h,1002 mbar,63,18µg/m³,22.5µg/m³,59.5µg/m³,6.5µg/m³,5µg/m³,Unknown,2022-12-09 14:56:35.857065


In [28]:
# función para sacar datos sobre la cantidad de arboles por provincia
# para obtener la información, debe hacer clic en el mapa cada vez que aparece un nuevo driver
# para obtener todos los datos del mapa, tuve que repetir el bucle muchas veces 
    # porque no encontraba la manera de obtener todos los datos a la vez.
# Cada vez que se abre el driver, obtiene la información sobre una provincia, 
    # pero no entendí cómo el sistema había terminado con la provincia específica cada vez.
# Me parecio que la provincia fue seleccionada al azar.
# Después de repetir el bucle muchas veces, terminé con información de casi todas las provincias.
# Todavía había algunas provincias para las que el programa no pudo obtener datos. 
# Para estas provincias, agregué la información al diccionario a mano.

def get_tree_count():
    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
    driver.maximize_window()
    driver.get('https://www.educaplus.org/geografia/es_prov_arboles.html')
    try:
        driver.find_element('xpath', '/html/body/div[1]/div/a[1]').click()
    except:
        pass
    tree_count = []
    pageSource = driver.find_element('xpath', '//*[@id="main"]').get_attribute("innerHTML")
    soup = BeautifulSoup(pageSource)
    gs = soup.find_all('g')
    for g in gs:
        try:
            texts = g.find('text')
        except:
            pass
    for t in texts:
        tree_count.append(t)
    return tree_count[-1].text 

In [32]:
tree_count_dict = {}
for n in range(10):
    try:
        prov_trees = get_tree_count()
        tree_count_dict[prov_trees.split(':')[0]] = prov_trees.split(':')[1]
    except:
        pass
tree_count_dict

C:\Users\Emily\anaconda3\envs\myenvsept\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


{'Cantabria': ' 122 millones',
 'Asturias': ' 228 millones',
 'León': ' 251 millones',
 'Lugo': ' 249 millones',
 'Burgos': ' 263 millones',
 'Córdoba': ' 85 millones'}

In [ ]:
# repito el bucle hasta tener los datos de todas las provincias.
for n in range(10):
    try:
        prov_trees = get_tree_count()
        tree_count_dict[prov_trees.split(':')[0]] = prov_trees.split(':')[1]
    except:
        pass
tree_count_dict

In [ ]:
# cuando el diccionario está listo, hago el dataframe
tree_count_df = pd.DataFrame(tree_count_dict, index=[0]).T
tree_count_df = tree_count_df.reset_index()
tree_count_df.columns = ['Provincia', 'Total_Trees']
tree_count_df

In [33]:
# función para sacar datos sobre población y km2 de cada provincia

h = urllib.request.urlopen('http://documentacion.diputacionalicante.es/ra_extension.asp')
soup = BeautifulSoup(h)
tds = []
for td in soup.find_all('td'):
    tds.append(td)
#[t.text for t in tds[:10]]
provs = []
for s in soup.find_all('td', class_="alt_celda"):
    if s.div == None:
        if len(s.text)>2:
            provs.append(s.text)
            
provs = []
for s in soup.find_all('td', class_="alt_celda"):
    if s.div == None:
        if len(s.text)>2:
            provs.append(s.text)
            
provs1 = []
for p in provs:
    if p == 'Fuente: Instituto Nacional de Estadística. España en Cifras\r\n      Fecha consulta: 09/12/2019':
        break
    provs1.append(p)
provs1.remove('41ª')
divs = soup.find_all('div', align='right')
km2 = []
pob = []
pob_densidad = []
for i in range(len(divs)):
    if i%3==0:
        km2.append(divs[i].text)
    if i%3==1:
        pob.append(divs[i].text)
    if i%3==2:
        pob_densidad.append(divs[i].text)

In [35]:
# crear dataframe de densidad de población con los datos anteriores
pob_densidad1 = []
for p in pob_densidad:
    # los numeros que vienen despues de 0,43 no son de densidad de poblacion.
    if p == '0,43':
        break
    pob_densidad1.append(p)
km2 = km2[:len(pob_densidad1)]
pob1 = [p for p in pob if len(p) > 2]
pob1 = pob1[:len(pob_densidad1)]
pob = pob[:len(pob_densidad)]
df_density = pd.DataFrame(columns = ['Area_km2', 'Population', 'Population_density'])
df_density['Area_km2'] = km2[1:]
df_density['Population'] = pob1[1:]
df_density['Population_density'] = pob_densidad1[1:]
df_density['Provinces'] = provs1
df_density.rename(columns = {'Population_density': 'densidad_habitantes_por_km2'}, inplace=True)
df_density = df_density[['Provinces', 'densidad_habitantes_por_km2', 'Area_km2', 'Population']]
df_density.Provinces[20] = 'Valencia/València'
df_density.Provinces[38] = 'Castellón/Castelló'
df_density.Provinces[48] = 'Araba/Álava'
df_density['densidad_habitantes_por_km2'] = [float(d.replace('.', '').replace(',','.')) for d in df_density['densidad_habitantes_por_km2']]
df_density.densidad_habitantes_por_km2[51] = df_density.densidad_habitantes_por_km2[51]/100
df_density.densidad_habitantes_por_km2[52] = df_density.densidad_habitantes_por_km2[52]/100
# df_density.to_csv('density.csv', index=False) 
df_density

C:\Users\Emily\anaconda3\envs\myenvsept\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Emily\anaconda3\envs\myenvsept\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Provinces,densidad_habitantes_por_km2,Area_km2,Population
0,TOTAL NACIONAL,92.300,505.979,46.722.980
1,Badajoz,31.100,21.766,676.376
2,Cáceres,20.000,19.868,396.487
3,Ciudad Real,25.200,19.813,499.100
4,Zaragoza,55.300,17.275,954.811
5,Cuenca,11.500,17.138,197.222
6,Huesca,14.000,15.636,219.345
7,León,29.800,15.578,463.746
8,Toledo,44.700,15.370,687.391
9,Albacete,26.000,14.926,388.786


In [44]:
# coches electricos por comunidad

h = urllib.request.urlopen('https://economia3.com/2022/03/31/475548-las-comunidades-autonomas-en-las-que-se-matriculan-mas-coches-electricos/')
soupCocheEl = BeautifulSoup(h)
lis = soupCocheEl.find_all('li')
communidades = []
coches_electricos = []
for li in lis:
    if li.find_all('strong'):
        communidades.append(li.text.split('(')[0].strip())
        coches_electricos.append(li.text.split('(')[1].replace(')', '').strip())
communidades, coches_electricos
coches_electricos_communidad_df = pd.DataFrame(columns = ['Communidad', 'No_coches_electricos'])
coches_electricos_communidad_df['Communidad'] = communidades
coches_electricos_communidad_df['No_coches_electricos'] = coches_electricos
coches_electricos_communidad_df['No_coches_electricos'] = [int(c.replace('.', '')) for c in coches_electricos_communidad_df.No_coches_electricos]

In [45]:
coches_electricos_communidad_df1 = coches_electricos_communidad_df[:19]
charging_stations_df = coches_electricos_communidad_df[19:]

In [46]:
coches_electricos_communidad_df1

,Communidad,No_coches_electricos
0,Comunidad de Madrid,5817
1,Cataluña,3261
2,Comunitat Valenciana,1108
3,Andalucía,1065
4,Canarias,700
5,País Vasco,461
6,Galicia,384
7,Castilla-La Mancha,326
8,Islas Baleares,317
9,Castilla y León,296


In [47]:
charging_stations_df

,Communidad,No_coches_electricos
19,Cataluña,3282
20,Comunidad de Madrid,1408
21,Comunitat Valenciana,1405
22,Andalucía,999
23,Islas Baleares,847
24,Canarias,645
25,Castilla y León,539
26,País Vasco,443
27,Región de Murcia,338
28,Galicia,330


In [55]:
# coches por 100 habitantes
        
# fuente de datos sobre coches: https://www.vulco.es/blog/sabes-cuantos-coches-por-habitante-tiene-tu-provincia/

# copy-paste de un programa que convierte jpg a texto:

coches_por_habitante =  """
Provincias
Balears (Illes)
1
2 Lugo
3 Ourense
4
Segovia
5 Pontevedra
6 Palencia
7 Zamora
8 Madrid
9 Coruña (A)
10 Leon
11 Cuenca
12 Soria
13 Ávila
14 Toledo
15 Girona
16 Santa Cruz de Tenerife
17 Cáceres
18 Badajoz
19 Teruel
20 Guadalajara
21 Alicante/Alacant
22 Burgos
23 Huesca
24 Castellón/Castelló
25 Cantabria
26 Lleida
27 Salamanca
Media
28 Melilla
29 Palmas (Las)
30 Tarragona
31 Valladolid
32 Murcia
33 Ceuta
34 Navarra
35 Asturias
36 Granada
37 Malaga
38 Almería
39 Valencia/València
40 Araba/Álava
41 Albacete
42 Ciudad Real
43 Sevilla
44 Huelva
45 Cordoba
46 Rioja (La)
47 Cadiz
48 Barcelona
49 Bizkaia
50 Gipuzkoa
51 Jaén
52 Zaragoza
Habitantes
1.107.220
336.527
314.853
155.652
944.346
85.818
512.529
164.644
89.304
180.406
97.426
6.466.996 3.442.362
1.122.799
597.015
473.604
251.206
201.071
106.492
90.040
47.577
162.514
85.853
688.672
362.831
753.576
1.004.124
403.665
684.113
136.977
252.882
1.836.459
360.995
221.079
579.245
582.206
434.041
335.985
792.299
523.679
1.464.847
Turismos Coches por 100 habitantes
684.714
61,84
57,71
57,30
55,13
54,27
54,24
54,00
53,23
53,17
53,04
52,96
52,84
52,83
52,69
52,13
51,97
51,72
51,70
51,47
51,45
51,16
51,05
50,84
50,75
50,60
50,34
50,17
49,93
49,84
49,80
84.519
640.647
1.042.608
194.205
180.419
86.026
42.876
1.097.800 546.700
387.735
255.963
715.588
41.154
311.091
502.479
439.877
780.203
704.297
336.853
2.544.264
1.205.761
324.126
152.907
392.118
183.043
506.888
236.393
1.939.775
894.387
519.596
237.318
791.610 357.785
315.794
140.413
1.239.889 546.133
5.542.680
2.437.180
1.147.576
717.832
648.250
950.507
915.392
1.629.298
392.868
521.846
208.764
353.684
70.507
130.108
939.545
184.275
112.399
293.984
294.606
218.488
168.559
504.417
314.097
276.708
402.385
48,94
48,88
48,85
48,69
48,56
48,19
48,05
47,89
47,83
47,39
47,18
46,68
46,64
46,11
45,67
45,20
44,46
44,05
43,97
43,95
43,76
42,69
42,33
""".split()

In [63]:
# crear dataframe de cantidad de coches por 100 habitantes
coches_total_y_pop_cien = ['684.714', '61,84', '57,71', '57,30', '55,13', '54,27', '54,24', '54,00', '53,23', '53,17', 
                            '53,04', '52,96', '52,84', '52,83', '52,69', '52,13', '51,97', '51,72', '51,70', '51,47', 
                            '51,45', '51,16', '51,05', '50,84', '50,75', '50,60', '50,34', '50,17', '49,93', '49,84', 
                            '49,80', '84.519', '640.647', '1.042.608', '194.205', '180.419', '86.026', '42.876', '1.097.800', 
                            '546.700', '387.735', '255.963', '715.588', '41.154', '311.091', '502.479', '439.877', '780.203', 
                            '704.297', '336.853', '2.544.264', '1.205.761', '324.126', '152.907', '392.118', '183.043', 
                            '506.888', '236.393', '1.939.775', '894.387', '519.596', '237.318', '791.610', '357.785', 
                            '315.794', '140.413', '1.239.889', '546.133', '5.542.680', '2.437.180', '1.147.576', '717.832', 
                            '648.250', '950.507', '915.392', '1.629.298', '392.868', '521.846', '208.764', '353.684',
                            '70.507', '130.108', '939.545', '184.275', '112.399', '293.984', '294.606', '218.488', '168.559',
                            '504.417', '314.097', '276.708', '402.385', '48,94', '48,88', '48,85', '48,69', '48,56', '48,19',
                            '48,05', '47,89', '47,83', '47,39', '47,18', '46,68', '46,64', '46,11', '45,67', '45,20', '44,46',
                            '44,05', '43,97', '43,95', '43,76', '42,69', '42,33']
provs2 = ['Balears', '1', '2', 'Lugo', '3', 'Ourense', '4', 'Segovia', '5', 'Pontevedra', '6', 'Palencia', '7', 'Zamora', 
          '8', 'Madrid', '9', 'Coruña', '(A)', '10', 'Leon', '11', 'Cuenca', '12', 'Soria', '13', 'Ávila', '14', 'Toledo', 
          '15', 'Girona', '16', 'Santa Cruz de Tenerife', '17', 'Cáceres', '18', 'Badajoz', '19', 'Teruel', '20', 
          'Guadalajara', '21', 'Alicante/Alacant', '22', 'Burgos', '23', 'Huesca', '24', 'Castellón/Castelló', '25', 
          'Cantabria', '26', 'Lleida', '27', 'Salamanca', 'Media', '28', 'Melilla', '29', 'Palmas', '(Las)', '30', 
          'Tarragona', '31', 'Valladolid', '32', 'Murcia', '33', 'Ceuta', '34', 'Navarra', '35', 'Asturias', '36', 
          'Granada', '37', 'Malaga', '38', 'Almería', '39', 'Valencia/València', '40', 'Araba/Álava', '41', 'Albacete', 
          '42', 'Ciudad Real', '43', 'Sevilla', '44', 'Huelva', '45', 'Cordoba', '46', 'Rioja', '(La)', '47', 'Cadiz', 
          '48', 'Barcelona', '49', 'Bizkaia', '50', 'Gipuzkoa', '51', 'Jaén', '52', 'Zaragoza']
l1 = []
for n in coches_total_y_pop_cien:
    if ',' in n:
        l1.append(n)
provs3 = []
for p in provs2:
    try:
        d = int(p)
    except:
        provs3.append(p)
provs3.remove('(A)')
provs3.remove('(Las)')
provs3.remove('(La)')
dfcoches = pd.DataFrame(columns=['provs', 'coches_por_100'])
dfcoches['provs'] = provs3
dfcoches['coches_por_100'] = l1
dfcoches['coches_por_100'] = [float(c.replace(',', '.')) for c in dfcoches['coches_por_100']]
dfcoches.provs[0] = 'Balears, Illes'
dfcoches.provs[47] = 'Cádiz'
dfcoches.provs[42] = 'Ciudad Real'
dfcoches.provs[8] = 'Coruña, A'
dfcoches.provs[9] = 'León'
dfcoches.provs[37] = 'Málaga'
dfcoches.provs[29] = 'Palmas, Las'
dfcoches.provs[37] = 'Rioja, La'
dfcoches.provs[45] = 'Córdoba'
dfcoches

C:\Users\Emily\anaconda3\envs\myenvsept\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Emily\anaconda3\envs\myenvsept\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Emily\anaconda3\envs\myenvsept\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Emily\anaconda3\envs\myenvsept\lib\site-packag

,provs,coches_por_100
0,"Balears, Illes",61.84
1,Lugo,57.71
2,Ourense,57.30
3,Segovia,55.13
4,Pontevedra,54.27
5,Palencia,54.24
6,Zamora,54.00
7,Madrid,53.23
8,"Coruña, A",53.17
9,León,53.04


In [64]:
# https://www.ine.es/jaxiT3/Datos.htm?t=2853
esperanza_de_vida_2021 = pd.read_csv('1485bsc.csv', encoding="ISO-8859-1", sep=';')
esperanza_de_vida_2021 = esperanza_de_vida[['Provincias', 'Total']]
esperanza_de_vida_2021['Provincias'] = [p[3:] for p in esperanza_de_vida_2021.Provincias]
esperanza_de_vida_2021['Total'] = [float(t.replace(',', '.')) for t in esperanza_de_vida_2021['Total']]
esperanza_de_vida_2021

,Provincias,Sexo,Periodo,Total
0,02 Albacete,Ambos sexos,2021,"83,22"
1,03 Alicante/Alacant,Ambos sexos,2021,"82,28"
2,04 Almería,Ambos sexos,2021,"80,65"
3,01 Araba/Álava,Ambos sexos,2021,"84,32"
4,33 Asturias,Ambos sexos,2021,"82,74"
5,05 Ávila,Ambos sexos,2021,"83,53"
6,06 Badajoz,Ambos sexos,2021,"81,87"
7,"07 Balears, Illes",Ambos sexos,2021,"83,36"
8,08 Barcelona,Ambos sexos,2021,"83,48"
9,48 Bizkaia,Ambos sexos,2021,"83,68"


In [66]:
# https://www.ine.es/jaxiT3/Datos.htm?t=3996

empleo_spain_2022T2 = pd.read_csv('empleo_espana2022T2.csv', sep=';')
empleo_spain_2022T2['Tasa de paro de la población'] = [float(n.replace(',', '.')) for n in empleo_spain_2022T2['Tasa de paro de la población']]
empleo_spain_2022T2['Tasa de empleo de la población'] = [float(n.replace(',', '.')) for n in empleo_spain_2022T2['Tasa de empleo de la población']]
empleo_spain_2022T2['Tasa de actividad'] = [float(n.replace(',', '.')) for n in empleo_spain_2022T2['Tasa de actividad']]
del empleo_spain_2022T2['Unnamed: 4']
empleo_spain_2022T2['Provincia'] = [p[3:] for p in empleo_spain_2022T2['Provincia']]
empleo_spain_2022T2

,Provincia,Tasa de actividad,Tasa de paro de la población,Tasa de empleo de la población
0,01 Total Nacional,58.71,12.48,51.38
1,02 Albacete,58.07,12.41,50.86
2,03 Alicante/Alacant,57.08,14.13,49.01
3,04 Almería,61.31,15.39,51.87
4,01 Araba/Álava,57.62,9.42,52.19
5,33 Asturias,48.86,11.36,43.31
6,05 Ávila,52.78,14.86,44.94
7,06 Badajoz,56.60,16.14,47.46
8,"07 Balears, Illes",65.42,9.29,59.34
9,08 Barcelona,61.29,9.18,55.67


In [69]:
# Fuente: INE
esperanza_de_vida = pd.read_csv('life_expectancy.csv')

In [70]:
esperanza_de_vida

,Provincias,Total,comunidad
0,Albacete,83.22,Castilla-La Mancha
1,Alicante/Alacant,82.28,Alicante/Alacant
2,Almería,80.65,Andalucía
3,Araba/Álava,84.32,Araba/Álava
4,Asturias,82.74,Asturias
5,Ávila,83.53,Castilla y León
6,Badajoz,81.87,Extremadura
7,"Balears, Illes",83.36,"Balears, Illes"
8,Barcelona,83.48,Cataluña
9,Bizkaia,83.68,Bizkaia


In [73]:
# fuente: INE
old_cars_km = pd.read_csv('old_cars.csv')
old_cars_km

,Comunidades,No_contesta,Total_vehículos,0_a_4_años,11_a_20_años,21_a_50_años,5_a_10_años
0,Andalucía,3367.2,10126.6,11698.0,7526.5,3580.1,9966.1
1,Aragón,0.0,11213.4,13105.0,8178.5,5672.5,11201.7
2,Asturias,3000.0,11238.3,12316.0,9205.1,9626.0,11431.4
3,"Balears, Illes",0.0,11370.6,12150.2,8169.3,4430.5,11503.6
4,Canarias,0.0,19585.7,20660.0,19433.9,17995.4,18799.2
5,Cantabria,4000.0,12897.5,13576.4,7199.6,844.5,14197.4
6,Castilla y León,3889.9,11361.5,13562.0,8713.7,4700.3,12099.8
7,Castilla-La Mancha,2000.0,12691.8,14688.6,9280.9,9866.4,13596.2
8,Cataluña,9554.6,11577.8,12623.5,8675.3,9653.4,11654.0
9,Ceuta,0.0,24024.4,15748.4,19801.2,5000.0,33299.4


In [ ]:
# despues de limpiar los datos, he guardado todos en csvs y en un database de sql.
# A traves del database, analizo y visualizo los datos en otro notebook. 